# Bulbauser

In [15]:
import mwparserfromhell

# Charger le fichier contenant l'infobox
with open("bulbasaur_infobox.txt", "r", encoding="utf-8") as file:
    wikitext = file.read()

# Parser le wikitext avec mwparserfromhell
wikicode = mwparserfromhell.parse(wikitext)

# Trouver le modèle d'infobox
templates = wikicode.filter_templates()
infobox = None
for template in templates:
    if template.name.matches("Pokémon Infobox"):
        infobox = template
        break

# Vérifier que l'infobox est trouvée
if infobox:
    print("Infobox trouvée !")
else:
    print("Infobox non trouvée.")


Infobox trouvée !


In [16]:
if infobox:
    # Extraire les paramètres et leurs valeurs
    data = {param.name.strip_code(): param.value.strip_code() for param in infobox.params}
    
    # Afficher les données extraites
    for key, value in data.items():
        print(f"{key}: {value}")


name: Bulbasaur
jname: フシギダネ
tmname: Fushigidane
ndex: 0001
type1: Grass
type2: Poison
category: Seed
height-ftin: 2'04"
height-m: 0.7
weight-lbs: 15.2
weight-kg: 6.9
abilityn: d
ability1: Overgrow
abilityd: Chlorophyll
egggroupn: 2
egggroup1: Monster
egggroup2: Grass
eggcycles: 20
evtotal: 1
evsa: 1
expyield: 64
oldexp: 64
lv100exp: 1,059,860
gendercode: 31
color: Green
catchrate: 45
body: 08
generation: 1
pokefordex: bulbasaur
friendship: 70


In [18]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD


# Initialiser le graphe RDF
g = Graph()

# Définir des namespaces
POKEMON = Namespace("http://example.org/pokemon#")
SCHEMA = Namespace("http://schema.org/")

# Ajouter les namespaces au graphe
g.bind("pokemon", POKEMON)
g.bind("schema", SCHEMA)

# URI de l'entité
bulbasaur = URIRef(POKEMON["Bulbasaur"])

# Ajouter des triples
g.add((bulbasaur, RDF.type, POKEMON.Pokemon))
g.add((bulbasaur, SCHEMA.name, Literal(data["name"], lang="en")))
g.add((bulbasaur, POKEMON.japaneseName, Literal(data["jname"], lang="ja")))
g.add((bulbasaur, POKEMON.type1, Literal(data["type1"], lang="en")))
g.add((bulbasaur, POKEMON.type2, Literal(data["type2"], lang="en")))
g.add((bulbasaur, POKEMON.height, Literal(data["height-m"], datatype=SCHEMA.Float)))
g.add((bulbasaur, POKEMON.height, Literal(data["height-m"], datatype=XSD.float)))
g.add((bulbasaur, POKEMON.weight, Literal(data["weight-kg"], datatype=XSD.float)))


# Sauvegarder le graphe dans un fichier Turtle
with open("bulbasaur.ttl", "w", encoding="utf-8") as rdf_file:
    rdf_file.write(g.serialize(format="turtle"))


# list of pokemon

In [19]:
import re
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD

# Chargement du fichier
with open("pokemon_list.txt", "r", encoding="utf-8") as file:
    content = file.read()

# Expression régulière pour capturer les données des Pokémon
pattern = r"\{\{rdex\|(\d+)\|\d+\|([\w\s'♂♀-]+)\|(\d+)\|([\w\s]+)\|([\w\s]*)\|gen=PE\}\}"
matches = re.findall(pattern, content)

# Vérification des données extraites
for match in matches[:5]:  # Affiche les 5 premiers résultats
    print(match)


('001', 'Bulbasaur', '2', 'Grass', 'Poison')
('002', 'Ivysaur', '2', 'Grass', 'Poison')
('003', 'Venusaur', '2', 'Grass', 'Poison')
('006', 'Charizard', '2', 'Fire', 'Flying')
('012', 'Butterfree', '2', 'Bug', 'Flying')


In [24]:
import re
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD
from urllib.parse import quote

# Charger le fichier contenant les Pokémon
try:
    with open("pokemon_list.txt", "r", encoding="utf-8") as file:
        content = file.read()
except FileNotFoundError:
    raise FileNotFoundError("Le fichier 'pokemon_list.txt' est introuvable. Vérifiez le chemin.")

# Expression régulière pour extraire les informations
pattern = r"\{\{rdex\|(\d+)\|\d+\|([\w\s'♂♀-]+)\|(\d+)\|([\w\s]+)\|([\w\s]*)\|gen=PE\}\}"
matches = re.findall(pattern, content)

if not matches:
    raise ValueError("Aucune correspondance trouvée. Vérifiez l'expression régulière ou la structure du fichier.")

# Initialiser le graphe RDF
g = Graph()

# Définir et lier les namespaces
POKEMON = Namespace("http://example.org/pokemon#")
SCHEMA = Namespace("http://schema.org/")
g.bind("pokemon", POKEMON)
g.bind("schema", SCHEMA)

# Fonction pour encoder proprement les URI
def create_uri(name):
    return URIRef(POKEMON[quote(name.replace(" ", "_").replace("'", "_").replace("♂", "Male").replace("♀", "Female"))])

# Générer des triples RDF
for match in matches:
    pokedex_number, name, stage, type1, type2 = match
    type2 = type2.strip()  # Nettoyer les espaces inutiles
    
    # Créer une URI pour le Pokémon
    pokemon_uri = create_uri(name)
    
    # Ajouter les triples RDF
    g.add((pokemon_uri, RDF.type, POKEMON.Pokemon))
    g.add((pokemon_uri, POKEMON.pokedexNumber, Literal(int(pokedex_number), datatype=XSD.integer)))
    g.add((pokemon_uri, SCHEMA.name, Literal(name, lang="en")))
    g.add((pokemon_uri, POKEMON.type1, Literal(type1, lang="en")))
    if type2:
        g.add((pokemon_uri, POKEMON.type2, Literal(type2, lang="en")))

# Vérification des namespaces liés au graphe
print("Namespaces enregistrés :")
for prefix, namespace in g.namespace_manager.namespaces():
    print(f"{prefix}: {namespace}")

# Sauvegarder le fichier RDF
output_file = "kanto_pokemon.ttl"
try:
    with open(output_file, "w", encoding="utf-8") as rdf_file:
        rdf_file.write(g.serialize(format="turtle"))
    print(f"Fichier RDF généré avec succès : {output_file}")
except Exception as e:
    print(f"Erreur lors de la sauvegarde du fichier RDF : {e}")


Namespaces enregistrés :
brick: https://brickschema.org/schema/Brick#
csvw: http://www.w3.org/ns/csvw#
dc: http://purl.org/dc/elements/1.1/
dcat: http://www.w3.org/ns/dcat#
dcmitype: http://purl.org/dc/dcmitype/
dcterms: http://purl.org/dc/terms/
dcam: http://purl.org/dc/dcam/
doap: http://usefulinc.com/ns/doap#
foaf: http://xmlns.com/foaf/0.1/
geo: http://www.opengis.net/ont/geosparql#
odrl: http://www.w3.org/ns/odrl/2/
org: http://www.w3.org/ns/org#
prof: http://www.w3.org/ns/dx/prof/
prov: http://www.w3.org/ns/prov#
qb: http://purl.org/linked-data/cube#
schema: https://schema.org/
sh: http://www.w3.org/ns/shacl#
skos: http://www.w3.org/2004/02/skos/core#
sosa: http://www.w3.org/ns/sosa/
ssn: http://www.w3.org/ns/ssn/
time: http://www.w3.org/2006/time#
vann: http://purl.org/vocab/vann/
void: http://rdfs.org/ns/void#
wgs: https://www.w3.org/2003/01/geo/wgs84_pos#
owl: http://www.w3.org/2002/07/owl#
rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs: http://www.w3.org/2000/01/rdf-sc

In [31]:
import re

# Lire le fichier texte
with open("pokemon_list.txt", "r", encoding="utf-8") as file:
    content = file.read()

# Expression régulière pour extraire les blocs {{rdex|...}}
pattern = r"{{rdex\|(\d+)\|(\d+)\|([\w\s'♂♀.-]+)\|(\d+)\|([\w]+)(?:\|([\w]+))?\|gen=PE(?:\|form=([A-Z]))?}}"
matches = re.findall(pattern, content)

# Générer des triplets RDF en Turtle
rdf_data = []

# Préfixes RDF
rdf_data.append("""
@prefix pokemon: <http://example.org/pokemon#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix bulbapedia: <https://bulbapedia.bulbagarden.net/wiki/> .

""")

# Générer des triplets pour chaque Pokémon
for match in matches:
    national_id = int(match[0])  # Numéro National
    regional_id = int(match[1])  # Numéro Régional
    name = match[2].strip()  # Nom du Pokémon
    stage = int(match[3])  # État d'évolution
    type1 = match[4].strip()  # Type principal
    type2 = match[5].strip() if match[5] else None  # Type secondaire (s'il existe)
    form = match[6].strip() if match[6] else None  # Forme alternative (s'il existe)

    # Identifiant unique pour le Pokémon
    sanitized_name = (
        name.replace(" ", "_")
        .replace("♂", "Male")
        .replace("♀", "Female")
        .replace("'", "")
    )
    pokemon_uri = f"pokemon:{sanitized_name}"

    # Ajouter les triplets RDF
    rdf_data.append(f"{pokemon_uri} a pokemon:Pokemon ;")
    rdf_data.append("    pokemon:usesTemplate pokemon:PokemonTemplate ;")
    rdf_data.append(f"    pokemon:nationalID \"{national_id}\"^^xsd:integer ;")
    rdf_data.append(f"    pokemon:regionalID \"{regional_id}\"^^xsd:integer ;")
    rdf_data.append(f"    rdfs:label \"{name}\" ;")
    rdf_data.append(f"    pokemon:stage \"{stage}\"^^xsd:integer ;")
    rdf_data.append(f"    pokemon:type1 \"{type1}\" ;")
    if type2:
        rdf_data.append(f"    pokemon:type2 \"{type2}\" ;")
    if form:
        rdf_data.append(f"    pokemon:form \"{form}\" ;")
    rdf_data.append(f"    owl:sameAs bulbapedia:{sanitized_name} .\n")

# Sauvegarder dans un fichier .ttl
with open("kanto_pokemon.ttl", "w", encoding="utf-8") as ttl_file:
    ttl_file.write("\n".join(rdf_data))

print("Fichier RDF généré : kanto_pokemon.ttl")


Fichier RDF généré : kanto_pokemon.ttl


In [28]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS, XSD, OWL

# Définir les namespaces
POKEMON = Namespace("http://example.org/pokemon#")
BULBAPEDIA = Namespace("https://bulbapedia.bulbagarden.net/wiki/")
EXAMPLE = Namespace("http://example.org/template#")

# Créer un graphe RDF
g = Graph()

# Ajouter des préfixes
g.bind("pokemon", POKEMON)
g.bind("bulbapedia", BULBAPEDIA)
g.bind("owl", OWL)
g.bind("xsd", XSD)

# Base du template
template_uri = EXAMPLE["PokemonInfoboxTemplate"]

# Ajouter les propriétés du template avec des placeholders
g.add((template_uri, RDF.type, OWL.Class))
g.add((template_uri, RDFS.label, Literal("Pokémon Infobox Template", lang="en")))

# Propriétés communes dans le template
properties = {
    "name": "Name of the Pokémon",
    "jname": "Japanese name",
    "tmname": "Trademarked name",
    "category": "Pokémon category",
    "ndex": "National Pokédex number",
    "type1": "Primary type",
    "type2": "Secondary type",
    "ability1": "First ability",
    "ability2": "Second ability",
    "abilityd": "Hidden ability",
    "eggGroup1": "First egg group",
    "eggGroup2": "Second egg group",
    "height": "Height of the Pokémon",
    "weight": "Weight of the Pokémon",
    "generation": "Generation in which Pokémon was introduced",
}

# Ajouter les propriétés dans le graphe
for prop, desc in properties.items():
    prop_uri = POKEMON[prop]
    g.add((prop_uri, RDF.type, OWL.DatatypeProperty))
    g.add((prop_uri, RDFS.label, Literal(prop, lang="en")))
    g.add((prop_uri, RDFS.comment, Literal(desc, lang="en")))
    g.add((prop_uri, RDFS.domain, template_uri))
    g.add((prop_uri, RDFS.range, XSD.string))

# Ajouter owl:sameAs pour Bulbapedia
same_as_uri = POKEMON["sameAs"]
g.add((same_as_uri, RDF.type, OWL.ObjectProperty))
g.add((same_as_uri, RDFS.label, Literal("sameAs", lang="en")))
g.add((same_as_uri, RDFS.comment, Literal("Link to Bulbapedia page", lang="en")))
g.add((same_as_uri, RDFS.domain, template_uri))
g.add((same_as_uri, RDFS.range, BULBAPEDIA["Pokemon"]))

# Sauvegarder le graphe en format Turtle
output_file = "pokemon_infobox_template.ttl"
g.serialize(destination=output_file, format="turtle")

print(f"Fichier RDF généré : {output_file}")


Fichier RDF généré : pokemon_infobox_template.ttl


In [4]:
def replace_rdfs_with_schema(input_file, output_file):
    """
    Replace all occurrences of rdfs:label with schema:name in an RDF Turtle file.

    :param input_file: Path to the input Turtle file
    :param output_file: Path to the output Turtle file
    """
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    updated_lines = []
    for line in lines:
        # Replace rdfs:label with schema:name
        updated_line = line.replace("rdfs:label", "schema:name")
        updated_lines.append(updated_line)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.writelines(updated_lines)

# Example usage
input_file = "pokemon_infobox.ttl"
output_file = "pokemon_infobox.ttl"
replace_rdfs_with_schema(input_file, output_file)


In [6]:
import re

def link_specific_pokemon_abilities(input_file, output_file):
    """
    Update the RDF file to link specific abilities to the ability namespace.

    Args:
    - input_file: Path to the input RDF file.
    - output_file: Path to the output RDF file.
    """
    # Define the abilities we want to process
    abilities_to_process = [
        "ability1", "ability2", "abilitym", "abilitym2",
        "abilityd", "abilityd2", "ability2-1", "ability2-2",
        "ability3-1", "ability3-2"
    ]
    
    # Create a regex pattern to match only the specified abilities
    ability_pattern = re.compile(
        r'pkmn:(' + '|'.join(re.escape(ability) for ability in abilities_to_process) + r')\s+"([^"]+)"'
    )

    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    updated_lines = []
    for line in lines:
        match = ability_pattern.search(line)
        if match:
            ability_key = match.group(1)
            ability_value = match.group(2).replace(" ", "_")
            # Replace the matched ability with a link to the ability namespace
            new_line = line.replace(
                f'pkmn:{ability_key} "{match.group(2)}"',
                f'pkmn:{ability_key} ability:{ability_value}'
            )
            updated_lines.append(new_line)
        else:
            updated_lines.append(line)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.writelines(updated_lines)

# Example usage
input_file = "pokemon_infobox.ttl"
output_file = "pokemon_infobox.ttl"
link_specific_pokemon_abilities(input_file, output_file)


In [ ]:
!pip install pyshacl

In [37]:
from pyshacl import validate

# Chargement des fichiers RDF et SHACL
rdf_file = "pokemon_infobox.ttl"
shacl_file = "pokemon_infobox_template.ttl"

# Validation SHACL
try:
    conforms, report_graph, report_text = validate(
        data_graph=rdf_file,
        shacl_graph=shacl_file,
        inference='rdfs',
        abort_on_first=False,
        allow_warnings=True,
    )

    # Affichage des résultats
    print("Validation conforme :", conforms)
    print("Rapport de validation :")
    print(report_text)

    # Sauvegarder le rapport
    with open("validation_report.ttl", "wb") as f:  # Mode binaire pour écrire les bytes
        f.write(report_graph)

    print("Rapport de validation sauvegardé dans 'validation_report.ttl'.")

except Exception as e:
    print("Une erreur s'est produite :", str(e))


Validation conforme : True
Rapport de validation :
Validation Report
Conforms: True

Une erreur s'est produite : a bytes-like object is required, not 'Graph'


In [43]:
from rdflib import Graph, Namespace, RDF, RDFS, URIRef

# Charger le fichier Turtle
input_file = "pokemon_infobox.ttl"
output_file = "pokemon_infobox.ttl"

g = Graph()
g.parse(input_file, format="turtle")

# Définir les namespaces
PKMN = Namespace("http://example.org/pokemon/")
POKEMON = Namespace("http://example.org/pokemon/pokemon/")
BULBAPEDIA = Namespace("https://bulbapedia.bulbagarden.net/wiki/")
OWL = Namespace("http://www.w3.org/2002/07/owl#")

# Ajouter les namespaces au graphe
g.bind("pokemon", POKEMON)
g.bind("pkmn", PKMN)
g.bind("bulbapedia", BULBAPEDIA)
g.bind("owl", OWL)

# Parcourir les ressources Pokémon
for pokemon in g.subjects(RDF.type, PKMN.Pokemon):
    # Récupérer le label du Pokémon pour construire l'URI Bulbapedia
    label = g.value(pokemon, RDFS.label)
    if label:
        # Créer un lien owl:sameAs avec le namespace "bulbapedia"
        bulbapedia_resource = BULBAPEDIA[str(label).replace(" ", "_")]
        g.add((pokemon, OWL.sameAs, bulbapedia_resource))

# Sauvegarder le graphe mis à jour
g.serialize(destination=output_file, format="turtle")

print(f"Mise à jour terminée. Fichier enregistré sous {output_file}")


Mise à jour terminée. Fichier enregistré sous pokemon_infobox.ttl


In [7]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_external_links(pokemon_name):
    """
    Récupère tous les liens externes pour un Pokémon donné, en convertissant les liens relatifs en absolus.
    
    Args:
        pokemon_name (str): Nom du Pokémon (ex: "Pikachu").
    
    Returns:
        list: Liste des URLs externes complètes.
    """
    base_url = "https://bulbapedia.bulbagarden.net/wiki/"
    formatted_name = pokemon_name.replace(" ", "_") + "_(Pokémon)"
    url = f"{base_url}{formatted_name}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Trouver la section "External links"
        external_links_section = soup.find("span", id="External_links")
        if not external_links_section:
            print(f"Section 'External links' introuvable pour {pokemon_name}.")
            return []
        
        # Récupérer les liens de la section "External links"
        links = []
        current_node = external_links_section.find_parent("h2").find_next_sibling()
        while current_node and current_node.name != "h2":
            # Rechercher les liens dans la section courante
            for link in current_node.find_all("a", href=True):
                # Convertir les liens relatifs en absolus
                absolute_url = urljoin(base_url, link['href'])
                links.append(absolute_url)
            current_node = current_node.find_next_sibling()

        return links
    
    except Exception as e:
        print(f"Erreur lors de la récupération des liens pour {pokemon_name}: {e}")
        return []


In [8]:
pikachu_links = get_external_links("Pikachu")
print("Liens externes pour Pikachu:", pikachu_links)


Liens externes pour Pikachu: ['https://en.wikipedia.org/wiki/Pikachu', 'https://bulbapedia.bulbagarden.net/wiki/Electric_rodents', 'https://bulbapedia.bulbagarden.net/wiki/Electric_rodents', 'https://bulbapedia.bulbagarden.net/wiki/Electric_rodents', 'https://bulbapedia.bulbagarden.net/wiki/Pikachu_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Raichu_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Generation_I', 'https://bulbapedia.bulbagarden.net/wiki/Pichu_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Generation_II', 'https://bulbapedia.bulbagarden.net/wiki/Plusle_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Minun_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Generation_III', 'https://bulbapedia.bulbagarden.net/wiki/Pachirisu_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Generation_IV', 'https://bulbapedia.bulbagarden.net/wiki/Emolga_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Generation_V', 'https:/

In [13]:
from rdflib import Graph, Namespace, RDF

def extract_pokemon_names(graph):
    """
    Extrait les noms des Pokémon depuis le fichier RDF.

    Args:
        graph (Graph): Graphe RDF contenant les données des Pokémon.

    Returns:
        list: Liste des noms des Pokémon.
    """
    pokemon_ns = Namespace("http://example.org/pokemon/")
    pokemon_names = []

    for s, p, o in graph.triples((None, RDF.type, pokemon_ns.Pokemon)):
        for _, _, name in graph.triples((s, pokemon_ns.name, None)):
            pokemon_names.append(str(name))

    return pokemon_names

# Charger le fichier RDF
input_file = 'pokemon_infobox.ttl'
g = Graph()
g.parse(input_file, format='turtle')

# Extraire les noms
pokemon_names = extract_pokemon_names(g)

# Sauvegarder dans un fichier texte
pokemon_list_file = 'pokemon_names.txt'
with open(pokemon_list_file, 'w', encoding='utf-8') as f:
    for name in pokemon_names:
        f.write(name + '\n')

pokemon_list_file


'pokemon_names.txt'

In [15]:
def extract_pokemon_names_from_text(input_file, output_file):
    """
    Extrait uniquement les noms des Pokémon depuis un fichier texte contenant des données brutes.
    
    Args:
        input_file (str): Chemin du fichier d'entrée contenant les données brutes.
        output_file (str): Chemin du fichier de sortie pour enregistrer les noms extraits.
    """
    pokemon_names = []
    
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            # Identifier les lignes contenant les données de Pokémon
            if line.startswith("{{rdex|"):
                # Extraire les champs séparés par "|"
                parts = line.split("|")
                if len(parts) > 3:
                    pokemon_name = parts[3].strip()  # Le nom du Pokémon est le 4ème champ
                    pokemon_names.append(pokemon_name)
    
    # Sauvegarder les noms dans un fichier de sortie
    with open(output_file, 'w', encoding='utf-8') as f:
        for name in pokemon_names:
            f.write(name + '\n')

# Fichiers d'entrée et de sortie
input_file = 'pokemon_list.txt'  # Fichier contenant les données brutes
output_file = 'pokemon_list.txt'  # Fichier de sortie pour les noms

# Exécuter l'extraction
extract_pokemon_names_from_text(input_file, output_file)

output_file


'pokemon_list.txt'

In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json

def get_pokemon_names(pokemon_list_file):
    """
    Lit un fichier contenant la liste des noms des Pokémon.
    
    Args:
        pokemon_list_file (str): Chemin du fichier contenant les noms des Pokémon.
    
    Returns:
        list: Liste des noms des Pokémon.
    """
    with open(pokemon_list_file, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

def get_external_links(pokemon_name):
    """
    Récupère tous les liens externes pour un Pokémon donné depuis Bulbapedia.

    Args:
        pokemon_name (str): Nom du Pokémon.

    Returns:
        list: Liste des URLs externes.
    """
    base_url = "https://bulbapedia.bulbagarden.net/wiki/"
    formatted_name = pokemon_name.replace(" ", "_") + "_(Pokémon)"
    url = f"{base_url}{formatted_name}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        # Trouver la section "External links"
        external_links_section = soup.find("span", id="External_links")
        if not external_links_section:
            return []

        # Récupérer les liens de la section "External links"
        links = []
        current_node = external_links_section.find_parent("h2").find_next_sibling()
        while current_node and current_node.name != "h2":
            for link in current_node.find_all("a", href=True):
                absolute_url = urljoin(base_url, link['href'])
                links.append(absolute_url)
            current_node = current_node.find_next_sibling()

        return links

    except Exception as e:
        print(f"Erreur lors de la récupération des liens pour {pokemon_name}: {e}")
        return []

def generate_pokemon_links_json(pokemon_list_file, output_json_file):
    """
    Génère un fichier JSON contenant les liens externes pour chaque Pokémon.

    Args:
        pokemon_list_file (str): Chemin du fichier contenant les noms des Pokémon.
        output_json_file (str): Chemin du fichier JSON de sortie.
    """
    pokemon_names = get_pokemon_names(pokemon_list_file)
    pokemon_links = {}

    for pokemon_name in pokemon_names:
        print(f"Processing: {pokemon_name}")
        links = get_external_links(pokemon_name)
        pokemon_links[pokemon_name] = links

    # Sauvegarder les résultats dans un fichier JSON
    with open(output_json_file, 'w', encoding='utf-8') as f:
        json.dump(pokemon_links, f, indent=4, ensure_ascii=False)

    print(f"Liens externes sauvegardés dans {output_json_file}")


In [ ]:
pokemon_list_file = 'pokemon_list.txt'
output_json_file = 'pokemon_external_links.json'
generate_pokemon_links_json(pokemon_list_file, output_json_file)


In [20]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS
import json

def add_external_links_to_rdf(rdf_file, json_file, output_file):
    """
    Ajoute les liens externes dans le fichier RDF pour chaque Pokémon en utilisant le fichier JSON.
    Utilise schema1:name pour identifier chaque Pokémon.

    Args:
        rdf_file (str): Chemin du fichier RDF contenant les données des Pokémon.
        json_file (str): Chemin du fichier JSON contenant les liens externes.
        output_file (str): Chemin du fichier RDF mis à jour.
    """
    # Charger le fichier RDF
    g = Graph()
    g.parse(rdf_file, format='turtle')

    # Charger les liens externes depuis le fichier JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        pokemon_links = json.load(f)

    # Namespace pour schema1 et externalLink
    schema1 = Namespace("http://schema.org/")
    external_link_predicate = URIRef("http://www.w3.org/2000/01/rdf-schema#externalLink")

    # Ajouter les liens externes à chaque Pokémon dans le RDF
    for pokemon_name, links in pokemon_links.items():
        # Identifier les ressources avec schema1:name correspondant au nom du Pokémon
        for s, p, o in g.triples((None, schema1.name, Literal(pokemon_name))):
            # Ajouter chaque lien externe sous rdfs:externalLink
            for link in links:
                g.add((s, external_link_predicate, URIRef(link)))

    # Sauvegarder le fichier RDF mis à jour
    g.serialize(destination=output_file, format='turtle')
    print(f"Liens externes ajoutés et fichier sauvegardé dans : {output_file}")

# Exemple d'utilisation
rdf_file = 'pokemon_infobox.ttl'  # Chemin du fichier RDF contenant les données des Pokémon
json_file = 'pokemon_external_links.json'  # Chemin du fichier JSON contenant les liens externes
output_file = 'pokemon_infobox_updated.ttl'  # Chemin du fichier RDF mis à jour

add_external_links_to_rdf(rdf_file, json_file, output_file)


Liens externes ajoutés et fichier sauvegardé dans : pokemon_infobox_updated.ttl


In [21]:
from rdflib import Graph, URIRef
from rdflib.namespace import RDFS

def remove_seealso_from_rdf(rdf_file, output_file):
    """
    Supprime toutes les triples contenant `rdfs:seeAlso` dans un fichier RDF.

    Args:
        rdf_file (str): Chemin du fichier RDF d'entrée.
        output_file (str): Chemin du fichier RDF mis à jour sans `rdfs:seeAlso`.
    """
    # Charger le fichier RDF
    g = Graph()
    g.parse(rdf_file, format='turtle')

    # Identifier et supprimer les triples avec rdfs:seeAlso
    seealso_predicate = RDFS.seeAlso
    triples_to_remove = [(s, p, o) for s, p, o in g.triples((None, seealso_predicate, None))]

    for triple in triples_to_remove:
        g.remove(triple)

    # Sauvegarder le fichier RDF mis à jour
    g.serialize(destination=output_file, format='turtle')
    print(f"Triples contenant `rdfs:seeAlso` supprimés. Fichier mis à jour sauvegardé dans : {output_file}")

# Exemple d'utilisation
rdf_file = 'pokemon_infobox_updated.ttl'  # Chemin du fichier RDF d'entrée
output_file = 'pokemon_infobox_updated.ttl'  # Chemin du fichier RDF mis à jour

remove_seealso_from_rdf(rdf_file, output_file)


Triples contenant `rdfs:seeAlso` supprimés. Fichier mis à jour sauvegardé dans : pokemon_infobox_updated.ttl


In [4]:
from rdflib import Graph, URIRef

# Chemin du fichier d'entrée et de sortie
file_path = "pokemon_infobox.ttl"  # Remplacez par le chemin de votre fichier local
output_path = "pokemon_infobox_corrected.ttl"  # Nom du fichier corrigé

# Charger le graphe RDF
graph = Graph()
graph.parse(file_path, format="ttl")

# Namespace pour bulbapedia et dbpedia
bulbapedia_prefix = "http://bulbapedia.bulbagarden.net/wiki/"
dbpedia_prefix = "http://dbpedia.org/resource/"

# Regrouper les triplets pour ne conserver qu'un seul lien par Pokémon
new_triples = set()

for s, p, o in graph.triples((None, URIRef("http://www.w3.org/2002/07/owl#sameAs"), None)):
    if str(o).startswith("bulbapedia:"):
        # Extraire le nom du Pokémon
        pokemon_name = str(o).split(":")[-1].split("_")[0]
        dbpedia_uri = URIRef(f"{dbpedia_prefix}{pokemon_name}")
        new_triples.add((s, p, dbpedia_uri))

# Supprimer tous les anciens liens owl:sameAs
for s, p, o in graph.triples((None, URIRef("http://www.w3.org/2002/07/owl#sameAs"), None)):
    graph.remove((s, p, o))

# Ajouter les nouveaux triplets owl:sameAs dbpedia
for triple in new_triples:
    graph.add(triple)

# Sauvegarder le fichier RDF corrigé
graph.serialize(destination=output_path, format="ttl")

print(f"Fichier corrigé sauvegardé dans : {output_path}")


Fichier corrigé sauvegardé dans : pokemon_infobox_corrected.ttl
